In [ ]:
from scipy import interpolate
from scipy.stats import norm
import collections
import numpy as np
import pandas as pd
import pdb
from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from plot_df_3d import plot_df_3d
from BSVol_to_LocVol import BSVol_to_LocVol
from Exotic_functions import *
from SVI_localvol import lvSPX
from tqdm import tqdm
import statsmodels.api as sm


In [ ]:
def LSV_impact_sim(rho=-0.01,nu=0.1,kappa=10):
    paths = 10000
    timesteps = 2000

    rho = rho
    nu = nu
    kappa = kappa 

    S0 = 1
    r = 0
    locvolMLP=lvSPX(mintau=1/250,S0=S0)
    delta_sigma = 0.03
    delta_S = 0.03
    beta=0.03

    k_array = np.linspace(-0.1, 0.05, 20)#log stock price log(St/S0)
    Strike_array=S0*np.exp(k_array)
    t_array = np.linspace(0, 1, len(k_array))
    t_array[0] = 1/250


    LVsurface=pd.DataFrame(index = k_array, columns = t_array,dtype=float)
    for t in t_array:
        for k in k_array:
            LVsurface.loc[k,t]=float(locvolMLP(S0*np.exp(k),t))

    # print(LVsurface)
    # plot_df_3d(LVsurface,title="local vol surface")
    volga_call_price_grid = pd.DataFrame(index = k_array, columns = t_array)
    sigma_kt_x_volga_kt_grid = pd.DataFrame(index = k_array, columns = t_array)
    x_volga_kt_grid = pd.DataFrame(index = k_array, columns = t_array)
    vanna_call_price_grid = pd.DataFrame(index = k_array, columns = t_array)
    sigma_kt_x_vanna_kt_grid = pd.DataFrame(index = k_array, columns = t_array)
    x_vanna_kt_grid = pd.DataFrame(index = k_array, columns = t_array)
    BS_imp_vol=pd.DataFrame(index = k_array, columns = t_array)

    Call_LV=pd.DataFrame(index = Strike_array, columns = t_array)
    Barrier_LV=pd.DataFrame(index = Strike_array, columns = t_array)
    Barrier_LSV=pd.DataFrame(index = Strike_array, columns = t_array)
    for t in t_array:
        print(t)
        c_kt_sigma = localVolMC(locvol = locvolMLP, S0 = S0, T = t,  paths = paths, timeSteps = timesteps, AK = Strike_array,kappa=kappa,nu=nu)
        c_kt_s_sigma_perturb = localVolMC(locvol = lambda u, t: locvolMLP(u,t) , S0 = S0 + delta_S, T = t,  paths = paths, timeSteps = timesteps, AK = Strike_array,kappa=kappa,nu=nu,deformation=delta_sigma)
        c_kt_sigmaperturb = localVolMC(locvol = lambda u, t: locvolMLP(u,t) , S0 = S0, T = t,  paths = paths, timeSteps = timesteps, AK = Strike_array,kappa=kappa,nu=nu,deformation=- delta_sigma)
        c_kt_s_perturb = localVolMC(locvol = locvolMLP, S0 = S0 + delta_S, T = t,  paths = paths, timeSteps = timesteps, AK = Strike_array,kappa=kappa,nu=nu)
        c_kt_negative_beta=localVolMC(locvol = locvolMLP, S0 = S0, T = t,  paths = paths, timeSteps = timesteps, AK = Strike_array,deformation=-beta,kappa=kappa,nu=nu)
        Call_LV[t]=c_kt_sigma['Local Volatility Price'].values
        volga_call_price_grid[t] = (2 * c_kt_sigma['Local Volatility Price'] - c_kt_negative_beta['Local Volatility Price']).values
        vanna_call_price_grid[t] = (c_kt_s_sigma_perturb['Local Volatility Price'] - c_kt_s_perturb['Local Volatility Price'] + c_kt_sigma['Local Volatility Price']).values
        
        sigma_kt_x_volga_kt_grid[t] = BSImpliedVolCall(S0, Strike_array, t, r, volga_call_price_grid[t].values)
        sigma_kt_x_vanna_kt_grid[t] = BSImpliedVolCall(S0, Strike_array, t, r, vanna_call_price_grid[t].values)
        BS_imp_vol[t]=c_kt_sigma['Black Scholes Implied Vol'].values
    # for k in k_array:
    #     for t in t_array:
    #         # change locvol to BS impvol
    #         x_volga_kt_grid.at[k,t] = sigma_kt_x_volga_kt_grid.at[k,t] - BS_imp_vol.at[k, t]
    #         x_vanna_kt_grid.at[k,t] = sigma_kt_x_vanna_kt_grid.at[k,t] - BS_imp_vol.at[k, t]
    # x_volga_kt_grid= sigma_kt_x_volga_kt_grid- BS_imp_vol
    # x_vanna_kt_grid= sigma_kt_x_vanna_kt_grid- BS_imp_vol

    volga_LocalVol=BSVol_to_LocVol(sigma_kt_x_volga_kt_grid)
    vanna_LocalVol=BSVol_to_LocVol(sigma_kt_x_vanna_kt_grid)

    volga_LocalVol_df=pd.DataFrame(index = k_array, columns = t_array).apply(lambda x:volga_LocalVol(x.index , x.name))
    vanna_LocalVol_df=pd.DataFrame(index = k_array, columns = t_array).apply(lambda x:vanna_LocalVol(x.index , x.name))

    x_volga_kt_grid= volga_LocalVol_df- LVsurface
    x_vanna_kt_grid= vanna_LocalVol_df- LVsurface
    # x_volga_kt_grid.iloc[:2,:2]=0
    # x_vanna_kt_grid.iloc[:2,:2]=0
    # plot_df_3d(x_volga_kt_grid.iloc[2:,2:],title="volga x_kt",ylabel='log strike')
    # plot_df_3d(x_vanna_kt_grid.iloc[2:,2:],title="vanna x_kt",ylabel='log strike')
    #t_interp = []
    #for _ in range(len(k_array)):
    #    t_interp.extend(t_array)
        
    x_volga_kt = interpolate.RectBivariateSpline(Strike_array, t_array, x_volga_kt_grid.astype(float).values)
    x_vanna_kt = interpolate.RectBivariateSpline(Strike_array, t_array, x_vanna_kt_grid.astype(float).values)
    # print(x_volga_kt_grid)
    # print(x_vanna_kt_grid)
    #x_volga_kt = interpolate.interp2d(np.repeat(k_array, len(t_array)), t_interp, [float(x) for x in x_volga_kt_grid.values.flatten()], kind='cubic')
    #x_vanna_kt = interpolate.interp2d(np.repeat(k_array, len(t_array)), t_interp, [float(x) for x in x_vanna_kt_grid.values.flatten()], kind='cubic')

    exotic_volga = pd.DataFrame(index = Strike_array, columns = t_array)
    exotic_vanna = pd.DataFrame(index = Strike_array, columns = t_array)

    for t in t_array:
        print(t)
        p_LSV=ExoticLSVMC(locvol = locvolMLP, S0 = S0, T = t,  paths = paths, timeSteps = timesteps, AK = Strike_array,kappa=kappa,nu=nu,rho=rho)

        p_kt_sigma_x_vanna = ExoticlocalVolMC(locvol = lambda u, t: locvolMLP(u,t) + x_vanna_kt(u,t,grid=False).flatten(), S0 = S0, T = t,  paths = paths, timeSteps = timesteps, AK = Strike_array,kappa=kappa,nu=nu) 
        p_kt_sigma_x_volga = ExoticlocalVolMC(locvol = lambda u, t: locvolMLP(u,t) + x_volga_kt(u,t,grid=False).flatten(), S0 = S0, T = t,  paths = paths, timeSteps = timesteps, AK = Strike_array,kappa=kappa,nu=nu) 
        p_kt_sigma = ExoticlocalVolMC(locvol = locvolMLP, S0 = S0, T = t,  paths = paths, timeSteps = timesteps, AK = Strike_array,kappa=kappa,nu=nu)
        p_kt_s_sigma_perturb = ExoticlocalVolMC(locvol = lambda u, t: locvolMLP(u,t) , S0 = S0 + delta_S, T = t,  paths = paths, timeSteps = timesteps, AK = Strike_array,kappa=kappa,nu=nu,deformation=delta_sigma)
        p_kt_sigmaperturb = ExoticlocalVolMC(locvol = lambda u, t: locvolMLP(u,t) , S0 = S0, T = t,  paths = paths, timeSteps = timesteps, AK = Strike_array,kappa=kappa,nu=nu,deformation=-delta_sigma)
        p_kt_s_perturb = ExoticlocalVolMC(locvol = locvolMLP, S0 = S0 + delta_S, T = t,  paths = paths, timeSteps = timesteps, AK = Strike_array,kappa=kappa,nu=nu)
        p_kt_negative_beta = ExoticlocalVolMC(locvol = locvolMLP, S0 = S0 , T = t,  paths = paths, timeSteps = timesteps, AK = Strike_array,deformation=-beta,kappa=kappa,nu=nu)
        
        Barrier_LSV[t]=p_LSV['Local Volatility Price'].values
        Barrier_LV[t]=p_kt_sigma['Local Volatility Price'].values
        exotic_volga[t] = (p_kt_sigma_x_volga['Local Volatility Price'].values - (2 * p_kt_sigma['Local Volatility Price'] - p_kt_negative_beta['Local Volatility Price']).values)/beta**2
        exotic_vanna[t] = (p_kt_s_sigma_perturb['Local Volatility Price'].values - p_kt_s_perturb['Local Volatility Price'].values + p_kt_sigma['Local Volatility Price'].values - p_kt_sigma_x_vanna['Local Volatility Price'].values)/(delta_S*delta_sigma)*S0


    # plot_df_3d(Call_LV,title='Call LV')
    # plot_df_3d(Barrier_LV,title='Barrier LV',zlabel="option price")
    # plot_df_3d(Barrier_LSV,title='Barrier LSV',zlabel="option price")

    # plot_df_3d(exotic_volga,title="Exotic Volga Barrier",zlabel="value")

    lsv_impact = (1 / 2 * (nu ** 2 / (2 * kappa)) * exotic_volga + rho * nu / kappa * exotic_vanna)
    # plot_df_3d(lsv_impact,title='LSV impact',zlabel="price")
    # plot_df_3d(Barrier_LSV-Barrier_LV,title='LSV model diff',zlabel="price")
    Barrier_LV.index=k_array
    Barrier_LSV.index=k_array
    lsv_impact.index=k_array
    return Barrier_LV,Barrier_LSV,lsv_impact

In [ ]:
Barrier_LV,Barrier_LSV,lsv_impact=LSV_impact_sim()

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,12)

for i in range(len(lsv_impact.columns)):
    plt.subplot(5,4,i+1)
    plt.plot(Barrier_LSV.index, lsv_impact.iloc[:,i],'b-',label='lsv impact')
    plt.plot(Barrier_LSV.index, Barrier_LSV.iloc[:,i]-Barrier_LV.iloc[:,i],'b--',label='actual lsv diff')
    plt.legend()
    plt.xlabel('log strike')
    plt.ylabel('price diff')
    plt.title('T='+'{0:.3}'.format(Barrier_LSV.columns[i]))
    plt.tight_layout()
plt.show()
# ax = lsv_impact.plot(subplots=True, layout=(5,4),xlabel='log strike',ylabel='price',label='lsv impact',style=['b-']*len(t_array))
# ax=ax.flatten()

# for i in range(len(ax)):
#     ax[i].plot(Barrier_LSV.index, Barrier_LSV.iloc[:,i]-Barrier_LV.iloc[:,i],'b--',label='actual impact')
#     ax[i].legend()



In [ ]:
def plot_LSV_impact(Barrier_LV,Barrier_LSV,lsv_impact):
    plt.rcParams["figure.figsize"] = (20,12)
    plt.rcParams['figure.dpi'] = 200
    times_periods=len(lsv_impact.columns)
    for i in range(times_periods):
        plt.subplot(int((times_periods-1e-5)/4)+1,4,i+1)
        plt.plot(Barrier_LSV.index, lsv_impact.iloc[:,i],'b-',label='lsv impact')
        plt.plot(Barrier_LSV.index, Barrier_LSV.iloc[:,i]-Barrier_LV.iloc[:,i],'b--',label='actual lsv diff')
        plt.legend()
        plt.xlabel('log strike')
        plt.ylabel('price diff')
        plt.title('T='+'{0:.3}'.format(Barrier_LSV.columns[i]))
        plt.tight_layout()
    plt.show()




In [ ]:
rhos=np.linspace(-.2,-0.01,3)

In [ ]:
rhos

In [ ]:
lv_list_rho=[]
lsv_list_rho=[]
lsv_impact_list_rho=[]
for rho in rhos:
    Barrier_LV,Barrier_LSV,lsv_impact=LSV_impact_sim(rho=rho)
    lv_list_rho.append(Barrier_LV)
    lsv_list_rho.append(Barrier_LSV)
    lsv_impact_list_rho.append(lsv_impact)
    Barrier_LV.to_csv('rho'+str(rho)+'LV.csv')
    Barrier_LV.to_csv('rho'+str(rho)+'LSV.csv')
    Barrier_LV.to_csv('rho'+str(rho)+'LSVimpact.csv')

In [ ]:
for i in range(len(rhos)):
    plot_LSV_impact(lv_list_rho[i],lsv_list_rho[i],lsv_impact_list_rho[i])

In [ ]:
kappas=[0.1,0.5,1,5,10]
lv_list_kappa=[]
lsv_list_kappa=[]
lsv_impact_list_kappa=[]
for kappa in kappas:
    Barrier_LV,Barrier_LSV,lsv_impact=LSV_impact_sim(kappa=kappa)
    lv_list_kappa.append(Barrier_LV)
    lsv_list_kappa.append(Barrier_LSV)
    lsv_impact_list_kappa.append(lsv_impact)
    Barrier_LV.to_csv('kappa'+str(kappa)+'LV.csv')
    Barrier_LV.to_csv('kappa'+str(kappa)+'LSV.csv')
    Barrier_LV.to_csv('kappa'+str(kappa)+'LSVimpact.csv')


In [ ]:
for i in range(len(kappas)):
    plot_LSV_impact(lv_list_kappa[i],lsv_list_kappa[i],lsv_impact_list_kappa[i])

In [ ]:
nus=[0.1,0.5]
lv_list_nu=[]
lsv_list_nu=[]
lsv_impact_list_nu=[]
for nu in nus:
    Barrier_LV,Barrier_LSV,lsv_impact=LSV_impact_sim(nu=nu)
    lv_list_nu.append(Barrier_LV)
    lsv_list_nu.append(Barrier_LSV)
    lsv_impact_list_nu.append(lsv_impact)
    Barrier_LV.to_csv('nu'+str(nu)+'LV.csv')
    Barrier_LV.to_csv('nu'+str(nu)+'LSV.csv')
    Barrier_LV.to_csv('nu'+str(nu)+'LSVimpact.csv')

In [ ]:
for i in range(len(nus)):
    plot_LSV_impact(lv_list_nu[i],lsv_list_nu[i],lsv_impact_list_nu[i])

In [ ]:
nus2=[0.1,0.5,1.0,5.0,10.0]
lv_list_nu_kappa=[]
lsv_list_nu_kappa=[]
lsv_impact_list_nu_kappa=[]
var_ratio=1000# var=1/2000=nu**2/kappa
for nu in nus2:
    kappa=var_ratio*nu**2
    Barrier_LV,Barrier_LSV,lsv_impact=LSV_impact_sim(nu=nu,kappa=kappa)
    lv_list_nu_kappa.append(Barrier_LV)
    lsv_list_nu_kappa.append(Barrier_LSV)
    lsv_impact_list_nu_kappa.append(lsv_impact)
    Barrier_LV.to_csv('nu{:.02}kappa{:.02}LV.csv'.format(nu,kappa))
    Barrier_LV.to_csv('nu{:.02}kappa{:.02}LSV.csv'.format(nu,kappa))
    Barrier_LV.to_csv('nu{:.02}kappa{:.02}LSVimpact.csv'.format(nu,kappa))
    

In [ ]:
for i in range(len(nus2)):
    print(nus2[i],var_ratio*nus2[i]**2)
    plot_LSV_impact(lv_list_nu_kappa[i],lsv_list_nu_kappa[i],lsv_impact_list_nu_kappa[i])

In [ ]:
def linear_reg(LV,LSV,LSVimpact):
    diff=LSV-LV
    diff=pd.DataFrame(diff.stack())
    diff['impact']=LSVimpact.stack()
    mod = sm.OLS(diff.iloc[:,0].values, diff.iloc[:,1].values)
    res = mod.fit()
    return res.params, res.pvalues

In [ ]:
len(lv_list_kappa)

In [ ]:
for i in range(len(rhos)):
    print(rhos[i])
    print(linear_reg(lv_list_rho[i],lsv_list_rho[i],lsv_impact_list_rho[i]))

In [ ]:
for i in range(len(kappas)):
    print(kappas[i])
    print(linear_reg(lv_list_kappa[i],lsv_list_kappa[i],lsv_impact_list_kappa[i]))

In [ ]:
for i in range(len(nus)):
    print(nus[i])
    print(linear_reg(lv_list_nu[i],lsv_list_nu[i],lsv_impact_list_nu[i]))

In [ ]:
for i in range(len(nus2)):
    print(nus2[i])
    print(linear_reg(lv_list_nu_kappa[i],lsv_list_nu_kappa[i],lsv_impact_list_nu_kappa[i]))